In [ ]:
import pickle
import numpy as np
import torch
import torch.nn as nn
from scipy.optimize import differential_evolution

# --- 1) Load your trained model and scalers ---

# Reconstruct the PyTorch architecture you used for training
def build_model(input_dim, hidden_dims, activation='ReLU'):
    act_key = activation.strip().lower()
    layers = []
    in_dim = input_dim
    for h in hidden_dims:
        layers.append(nn.Linear(in_dim, h))
        if act_key not in ('linear', 'none'):
            layers.append(getattr(nn, activation)())
        in_dim = h
    layers.append(nn.Linear(in_dim, 1))
    return nn.Sequential(*layers)

# Paths to your saved artifacts
MODEL_PATH    = 'trained_model_ab12cd34.pt'
SCALER_X_PATH = 'scaler_X.pkl'
SCALER_Y_PATH = 'scaler_y.pkl'

# Load model
hidden_dims = [64, 32]   # must match training
activation  = 'ReLU'
# we'll infer input_dim from scaler_X
with open(SCALER_X_PATH, 'rb') as f:
    scaler_X = pickle.load(f)
with open(SCALER_Y_PATH, 'rb') as f:
    scaler_y = pickle.load(f)

input_dim = sum(len(v) for v in {
    'Input 1': [1,2],
    'Input 2': [1,2,3,4,5],
    'Input 3': [1,2,3,4,5],
    'Input 4': list(range(1,18)),
    'Input 5': list(range(1,6)),
    'Input 6': [1,2,3],
    'Input 7': [1,2,3,4],
    'Input 8': list(range(1,11))
}.values()) + scaler_X.scale_.shape[0]

model = build_model(input_dim, hidden_dims, activation)
model.load_state_dict(torch.load(MODEL_PATH))
model.eval()

# --- 2) Definitions for inversion ---

# Fixed inputs (all except Input 10)
fixed_inputs = {
    'Input 1':  2,
    'Input 2':  3,
    'Input 3':  1,
    'Input 4': 10,
    'Input 5':  4,
    'Input 6':  2,
    'Input 7':  1,
    'Input 8':  7,
    'Input 9':  123.4,
    # 'Input 10' is what we’ll solve for
    'Input 11': 0.56,
    'Input 12':  78.9,
    'Input 13':   0.002,
    'Input 14': 12345.0,
    'Input 15':  200.0,
    'Input 16':  12.3,
    'Input 17': 4567.8,
    'Input 18':   9.1,
}

# Desired true output (in original scale)
desired_output =  25000.0

# Convert desired output to normalized scale
desired_norm = scaler_y.transform([[desired_output]]).ravel()[0]

# Fixed category definitions
categories_map = {
    'Input 1': [1, 2],
    'Input 2': [1, 2, 3, 4, 5],
    'Input 3': [1, 2, 3, 4, 5],
    'Input 4': list(range(1, 18)),
    'Input 5': list(range(1, 6)),
    'Input 6': [1, 2, 3],
    'Input 7': [1, 2, 3, 4],
    'Input 8': list(range(1, 11)),
}

# Build a single feature‐vector function given a candidate x10
def make_feature_vector(x10):
    # 1) Categorical one‐hots for Inputs 1–8
    cat_parts = []
    for i in range(1, 9):
        name = f'Input {i}'
        cats = categories_map[name]
        val  = fixed_inputs[name]
        # build one-hot in fixed order
        onehot = [1.0 if val == c else 0.0 for c in cats]
        cat_parts.extend(onehot)
    # 2) Numeric inputs 9–18 (including variable Input 10)
    num_names = [f'Input {i}' for i in range(9, 19)]
    num_vals  = []
    for nm in num_names:
        if nm == 'Input 10':
            num_vals.append(x10)
        else:
            num_vals.append(fixed_inputs[nm])
    num_arr = np.array(num_vals, dtype=np.float32).reshape(1, -1)
    # 3) Normalize numeric
    num_scaled = (num_arr - scaler_X.mean_) / scaler_X.scale_
    # 4) Concatenate
    X = np.hstack([np.array(cat_parts, dtype=np.float32).reshape(1, -1),
                   num_scaled.astype(np.float32)])
    return X

# Objective: squared error in normalized output
def objective(x):
    X = make_feature_vector(x[0])
    with torch.no_grad():
        pred_norm = model(torch.from_numpy(X)).cpu().numpy().ravel()[0]
    return (pred_norm - desired_norm)**2

# Bounds for Input 10 (choose appropriate range)
bounds = [(0.0, 1000.0)]   # adjust min/max to domain of Input 10

# --- 3) Run differential evolution ---
result = differential_evolution(objective, bounds, maxiter=200, tol=1e-6)
x10_opt = result.x[0]

# 4) Report
print(f"Optimal Input 10 (unscaled): {x10_opt:.6f}")

# Verify final prediction
X_opt = make_feature_vector(x10_opt)
with torch.no_grad():
    pred_norm = model(torch.from_numpy(X_opt)).cpu().numpy().ravel()[0]
pred_orig = scaler_y.inverse_transform([[pred_norm]])[0,0]
print(f"Predicted output at this Input 10: {pred_orig:.4f} "
      f"(target was {desired_output})")
